## Test data

In [1]:
import sys
sys.path.append('../')
from util import util, corpus, alignment, tokenize

from tqdm.notebook import tqdm as tq
import pandas as pd

import torch
import torch.nn.functional as F

[nltk_data] Downloading package stopwords to /home/sudhi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sudhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
test_df = util.decompress_pickle('../../data/test/test_df_1000_queries')
all_claims_en = util.decompress_pickle('../../data/test/test_en_claims_first_50_queries')
all_claims_de = util.decompress_pickle('../../data/test/test_de_claims_first_50_queries')

In [3]:
test_small_df = test_df[:50]
test_small_df.columns = ['name', 'query_title', 'rel_titles', 'irr_titles', 'own_claims_en',
       'own_claims_de', 'rel_claims_en', 'rel_claims_de', 'irr_claims_en',
       'irr_claims_de']
test_small_df.keys()

Index(['name', 'query_title', 'rel_titles', 'irr_titles', 'own_claims_en',
       'own_claims_de', 'rel_claims_en', 'rel_claims_de', 'irr_claims_en',
       'irr_claims_de'],
      dtype='object')

In [4]:
def remove_short_claims(en_claims, de_claims):
    en_claims_proc, de_claims_proc = [], []
    for en, de in zip(en_claims, de_claims):
        if len(en.split(' ')) <= 13:
            continue
        en_claims_proc.append(en)
        de_claims_proc.append(de)
    return en_claims_proc, de_claims_proc

In [5]:
test = []
for name, query_title, rel_titles, irr_titles, \
    own_claims_en, own_claims_de, \
    rel_claims_en, rel_claims_de, \
    irr_claims_en, irr_claims_de \
    in zip(test_small_df.name, \
           test_small_df.query_title,
           test_small_df.rel_titles,
           test_small_df.irr_titles,
           test_small_df.own_claims_en,
           test_small_df.own_claims_de,
           test_small_df.rel_claims_en,
           test_small_df.rel_claims_de,
           test_small_df.irr_claims_en,
           test_small_df.irr_claims_de
          ):
    own_en, own_de = remove_short_claims(own_claims_en, own_claims_de)
    rel_en, rel_de = remove_short_claims(rel_claims_en, rel_claims_de)
    irr_en, irr_de = remove_short_claims(irr_claims_en, irr_claims_de)
    
    test.append(
        {
            'name':name,
            'query_title':query_title,
            'rel_titles':rel_titles,
            'irr_titles':irr_titles,
            'own_claims_en':own_en,
            'own_claims_de':own_de,
            'rel_claims_en':rel_en,
            'rel_claims_de':rel_de,
            'irr_claims_en':irr_en,
            'irr_claims_de':irr_de          
            
        }
    )
    
test_final = pd.DataFrame(test)
test_final

,name,query_title,rel_titles,irr_titles,own_claims_en,own_claims_de,rel_claims_en,rel_claims_de,irr_claims_en,irr_claims_de
0,EP11787853NWB1.xml,"Wiper Blade Adapter, In Particular For A Motor...","[Wiper Blade Adapter, In Particular For A Moto...",[Driving Prediction With A Deep Neural Network...,"[Wiper blade adapter, in particular for a moto...","[Wischblattadapter, insbesondere für eine Kraf...","[Wiper blade adapter, in particular for a moto...","[Wischblattadapter, insbesondere für eine Kraf...",[Method for assisting a driver to drive a vehi...,[Verfahren zum Unterstützen eines Fahrers beim...
1,EP14001859NWB1.xml,Method For Producing A Suspension Arm And Susp...,[Method For Producing A Suspension Arm And Sus...,"[Caster Indicator, Tolerance Gap Flag, Psi5 In...",[Method for producing a suspension link 1 for ...,[Verfahren zur Herstellung eines Fahrwerkslenk...,[Method for producing a suspension link 1 for ...,[Verfahren zur Herstellung eines Fahrwerkslenk...,[A caster indicator 200 comprising plural leve...,"[Rollenanzeige 200, welche Folgendes aufweist ..."
2,EP14189324NWB1.xml,Removable Luggage Box For Storage In A Luggage...,[Removable Luggage Box For Storage In A Luggag...,[Illumination For Detecting Raindrops On A Pan...,[A motor vehicle 1 having a confined space 2 o...,"[Kraftfahrzeug 1 mit einem begrenzten Raum 2, ...",[A motor vehicle 1 having a confined space 2 o...,"[Kraftfahrzeug 1 mit einem begrenzten Raum 2, ...",[A device for detecting rain on a pane 2 compr...,[Vorrichtung zur Erkennung von Regen auf einer...
3,EP16725464NWB1.xml,Electrical Steering Column Lock,"[Electrical Steering Column Lock, A Steering C...","[Nonwoven Textile Made From Short Fibers, Biax...",[Electrical steering column lock for locking a...,[Elektrisches Lenkradschloss zum Verriegeln un...,[Electrical steering column lock for locking a...,[Elektrisches Lenkradschloss zum Verriegeln un...,"[A method of forming a nonwoven material, the ...",[Verfahren zur Herstellung eines Vliesmaterial...
4,EP16700859NWB1.xml,Hollow Profiled Element Having Depressions,[Hollow Profiled Element Having Depressions],[Method For Managing The Supply Of Electrical ...,"[A hollow profiled element 1, comprising a plu...",[Hohlprofil 1 mit mehreren in einer Längsricht...,"[A hollow profiled element 1, comprising a plu...",[Hohlprofil 1 mit mehreren in einer Längsricht...,[A method for managing the supply of electrica...,[Verfahren zur Verwaltung der Stromversorgung ...
5,EP12798981NWB1.xml,Method And A System For Combined Hydrogen And ...,[Method And A System For Combined Hydrogen And...,"[Tolerance Gap Flag, Child Protector Mechanism...",[A method of using a SOFC system 100 for produ...,[Verfahren zur Verwendung eines SOFC-Systems 1...,[A method of using a SOFC system 100 for produ...,[Verfahren zur Verwendung eines SOFC-Systems 1...,[Decorative arrangement 4 for mounting on a su...,[Dekorative Anordnung 4 zur Montage an einem T...
6,EP14807582NWB1.xml,System And Method For Applying A Lubricating P...,[System And Method For Applying A Lubricating ...,"[Child Protector Mechanism, Isolation Systems ...","[A system, comprising a processing station 16b...","[System, umfassend eine Verarbeitungsstation 1...","[A system, comprising a processing station 16b...","[System, umfassend eine Verarbeitungsstation 1...","[A child protector mechanism 10, comprising a ...","[Kindersicherungsmechanismus 10, der Folgendes..."
7,EP16160480NWB1.xml,Thermal Dissipation System Of An Electric Vehicle,[Thermal Dissipation System Of An Electric Veh...,"[Ball Joint, Stereo Camera, Fixation Of The Ba...",[A thermal dissipation system suitable for pla...,"[Wärmeableitungssystem, geeignet zum Platziere...",[A thermal dissipation system suitable for pla...,"[Wärmeableitungssystem, geeignet zum Platziere...",[A ball joint 1b comprising a ball stud 10 tha...,[Kugelgelenk 1b bestehend aus einem Kugelzapfe...
8,EP13703552NWB1.xml,Method For Producing An Operating Device In A ...,[Method For Pro

In [6]:
all_claims_en = []
all_claims_de = []

for query, own_claims_en, own_claims_de, \
    rel_claims_en, rel_claims_de in zip(test_final.query_title,
                                        test_final.own_claims_en,
                                        test_final.own_claims_de,
                                        test_final.rel_claims_en,
                                        test_final.rel_claims_de):
    all_claims_en.extend(own_claims_en)
    all_claims_en.extend(rel_claims_en)
    
    all_claims_de.extend(own_claims_de)
    all_claims_de.extend(rel_claims_de)    

In [7]:
def evaluate(encoding, num_ret, en_doc_vectors, de_doc_vectors): 
    prec_av, recall_av, fscore_av = 0, 0, 0
    
    for query, own_claims_en, own_claims_de, \
        rel_claims_en, rel_claims_de in zip(test_final.query_title,
                                            test_final.own_claims_en,
                                            test_final.own_claims_de,
                                            test_final.rel_claims_en,
                                            test_final.rel_claims_de):
        
        # print('query', query)
        
        relevant_claims = own_claims_en + own_claims_de + \
                            rel_claims_en + rel_claims_de
        
        # num_ret = int(len(relevant_claims)/2)
        # num_ret = 50
        out_en = get_retrieved_docs_test(
                                      encoding=encoding,
                                      query=query, 
                                      num_ret=num_ret, 
                                      docs=all_claims_en, 
                                      doc_vectors=en_doc_vectors, 
                                      lang='en')
        
        out_de = get_retrieved_docs_test(
                                      encoding=encoding,
                                      query=query, 
                                      num_ret=num_ret, 
                                      docs=all_claims_de, 
                                      doc_vectors=de_doc_vectors, 
                                      lang='en')
        
        retrieved_claims = out_en['docs'] + out_de['docs']
        
        relevant_retrieved = 0
        for claim in relevant_claims:
            if claim in retrieved_claims:
                relevant_retrieved += 1
        
        relevant = len(relevant_claims)
        retreived = len(retrieved_claims)
        
        prec = relevant_retrieved/retreived
        recall = relevant_retrieved/relevant     
    
        # print('P: %f R: %f' %(prec, recall))
    
        prec_av += prec
        recall_av += recall
        
    print('Average:\tP: %f R: %f' %(prec_av/len(test_small_df.query_title), 
                                recall_av/len(test_small_df.query_title)))

## Define encode function

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [9]:
import numpy as np

class Encoding:
    def __init__(self, model, vocab=None, 
                 do_pre_process=False,
                 do_noun_chunks=False,
                 do_dual_token=False,
                 do_dual_embed=False,
                 do_avg=False,
                 post_hoc_encode=False,
                 contextual_encode=False,
                 contextual_encode_full=False,
                 layers=False
                ):
        self.model = model
        self.vocab = vocab
        
        self.do_pre_process=do_pre_process
        self.do_noun_chunks=do_noun_chunks
        self.do_dual_token=do_dual_token
        self.do_dual_embed=do_dual_embed
        self.do_avg=do_avg
        
        self.post_hoc_encode=post_hoc_encode
        self.contextual_encode=contextual_encode
        self.contextual_encode_full=contextual_encode_full
        self.layers=layers
        
    def encode(self, sent, lang=None):
        if self.post_hoc_encode:
            return self.encode_post_hoc(sent, lang)
        elif self.contextual_encode:
            return torch.tensor(self.model.encode(sent, show_progress_bar=False))
        elif self.contextual_encode_full:
            return self.encode_cont(sent)
        else:
            return self.encode_with_alg(sent, lang)
    
    def encode_with_alg(self, sent, lang):
        pre = sent
        if self.do_pre_process:
            pre = pre_process_claim(pre, lang=lang)
        if self.do_noun_chunks:
            pre = find_and_replace_ncs(pre, lang=lang)
        if self.do_dual_token:
            tokens_pre = tokenize.get_tokens(pre, lang=lang)
            tokens_wo_pre = tokenize.get_tokens(sent, lang=lang)
            tokens = tokens_pre + tokens_wo_pre
        else:
            tokens = tokenize.get_tokens(sent, lang=lang)
            
        if self.do_dual_embed:
            word_vectors = [np.asarray(self.model.wv[token] + \
                                       self.model.syn1neg[self.vocab.index(token)]
                                      ) \
                            for token in tokens if token in self.vocab]
        else:
            word_vectors = [np.asarray(self.model.wv[token]) \
                            for token in tokens if token in self.vocab]
            
        if self.do_avg:
            sent_vector = torch.tensor(np.sum(word_vectors, axis=0)/len(tokens))
        else:
            sent_vector = torch.tensor(np.sum(word_vectors, axis=0))
            
        if sent_vector.size() == torch.Size([]):
            sent_vector = torch.zeros(300)
            
        return sent_vector
    
    
    def encode_post_hoc(self, sent, lang):
        model = self.model[0] if lang=='en' else self.model[1] 
        vocab = self.vocab[0] if lang=='en' else self.vocab[1]

        tokens = tokenize.get_tokens(sent, lang=lang)
        word_vectors = [np.asarray(model[token]) for token in tokens \
                        if token in vocab]
        
        if self.do_avg:
            sent_vector = torch.tensor(np.sum(word_vectors, axis=0)/len(word_vectors))
        else:
            sent_vector = torch.tensor(np.sum(word_vectors, axis=0))    
            
        if sent_vector.size() == torch.Size([]):
            sent_vector = torch.zeros(300)

        return sent_vector
    
    
    def encode_cont(self, doc):
        with torch.no_grad():
            inputs = tokenize(doc, return_tensors='pt',
                          max_length=512, truncation=True,
                          padding='max_length').to('cuda')
            hidden_states = self.model(**inputs, output_hidden_states=True)['hidden_states']

            # token_embeddings = hidden_states[-1] #First element of model_output contains all token embeddings
            token_embeddings = torch.sum(torch.cat(hidden_states[self.layers[0]:self.layers[1]]), 0).unsqueeze(0)
            attention_mask = inputs['attention_mask']
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            sent_vector = torch.sum(token_embeddings * input_mask_expanded, 1) / \
                                    torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sent_vector[0]

In [10]:
def get_retrieved_docs_test(encoding, query, num_ret, docs, doc_vectors, lang=None):
    # print('Query', query)
    query_vector = encoding.encode(sent=query, lang=lang)

    output = F.cosine_similarity(query_vector.unsqueeze(0), doc_vectors, dim=1)
    ret_ids = torch.topk(output, num_ret).indices.cpu().numpy()   
    
    ret_docs = [docs[idx] for idx in ret_ids]
    
    ret = {
            'doc_ids': ret_ids,
            'docs': ret_docs
        }
    
    return ret  

In [11]:
def vectorize_docs(encoding):
    en_doc_vectors = []
    for claim in tq(all_claims_en):
        en_doc_vectors.append(encoding.encode(claim, lang='en'))
        
    de_doc_vectors = []
    for claim in tq(all_claims_de):
        de_doc_vectors.append(encoding.encode(claim, lang='de'))
        
    en_doc_vectors = torch.stack(en_doc_vectors)
    de_doc_vectors = torch.stack(de_doc_vectors)        
        
    return en_doc_vectors, de_doc_vectors

In [13]:
epo_vocab_en = util.decompress_pickle('../../data/epo_vocab/vocab/vocab_en')
epo_vocab_de = util.decompress_pickle('../../data/epo_vocab/vocab/vocab_de')

In [14]:
algn_dic = util.decompress_pickle('../../data/epo_vocab/algn_dic')
# lower case all entries in the dictionary
algn_dic.en = algn_dic.en.str.lower()
algn_dic.de = algn_dic.de.str.lower()
# remove entries not in vocab
algn_dic = algn_dic[algn_dic.de.isin(epo_vocab_de)]
algn_dic = algn_dic[algn_dic.en.isin(epo_vocab_en)]

In [15]:
algn_dic

,en,de,prob
0,rotating_side_member,rotierenden_seitenelements,1
1,visible_interface,sichtbare_grenzfläche,1
2,current_density,stromdichte,1
4,novolac_epoxy_resin,epoxyharz,0.982917
6,maximum_charging_voltage,zweite_batteriechemikalie,0.37695
...,...,...,...
166896,toughness,zähigkeit,0.666667
166897,increasing_speed,steigender_geschwindigkeit,0.501577
166898,corresponding_threshold_value,entsprechenden_schwellwert,0.888865
166899,loading_site,beladeort,0.666667


In [16]:
import re
from nltk.corpus import wordnet as wn
import spacy
from spacy.matcher import Matcher
nlp_en = spacy.load("en_core_web_lg")
nlp_de = spacy.load("de_core_news_lg")

In [17]:
EN_CORR_KEYS = [\
                'and', 
                'whereas',
                'wherein',
                'least',
                'that',
                'by',
                'particular',
                'there',
                'characterized',
               ]

DE_CORR_KEYS = [\
                'und', 
                'wohingegen',
                'wherein',
                'worin',
                'dass',
                'von',
                'characterized',
                'gekennzeichnet',
                'der'
               ]


In [18]:
def add_space_between_words(key, word):
    # correct words starting with key word and no space
    corr_word = ''
    if re.search(r'^%s'%key, word):
        if '-' in word:
            corr_word = ''
            pass
        elif not wn.synsets(word):
            corr_word = re.sub(r'^%s'%key, '%s '%key, word)
            # print(word, '-->', corr_word)
    return corr_word

In [19]:
incorrect_words_en = {}
for word in tq(epo_vocab_en):
    for key in EN_CORR_KEYS:
        corr_word = add_space_between_words(key, word)
        if corr_word != '':
            # print(corr_word)
            incorrect_words_en[word] = corr_word
            
            
incorrect_words_de = {}
for word in tq(epo_vocab_de):
    for key in DE_CORR_KEYS:
        corr_word = add_space_between_words(key, word)
        if corr_word != '':
            incorrect_words_de[word] = corr_word            

In [20]:
def pre_process_claim(claim, lang):
    # add space between punctuation and the word
    # ,wherein --> , wherein
    # claim = re.sub(r'([,;./:])([a-zA-Z])', r' \2', claim)
    # claim = re.sub(r'([a-zA-Z])([,;./:])', r'\1 \2', claim)

    # remove punctuations except for :;
    claim = re.sub(r'[!"#$%&\()*+/<=>?@\[\\\\\]^_`{|}~]', '', claim)
    
    c_s = claim.split(' ')
    
    if lang == 'en':
        # replace claims references to [CLAIMS]
        # claims 1 to 2 --> [CLAIMS]
        # claim = re.sub(r'[c|C]laims\s\d{1,}\s[a-z]{1,}\s\d{1,}', '', claim)
        # # claim 1 to 2 --> [CLAIMS]
        # claim = re.sub(r'[c|C]laim\s\d{1,}\s[a-z]{1,}\s\d{1,}', '', claim)
        # # claim 1 --> [CLAIMS]
        # claim = re.sub(r'[c|C]laim\s\d{1,}', '', claim)
        # claim = re.sub(r'andan', 'and an', claim)
        # claim = re.sub(r'[0-9]{1,}', 'NUM', claim)
        for s in c_s:
            if s in incorrect_words_en.keys():
                # print(s)
                claim = re.sub(s, incorrect_words_en[s], claim)

    elif lang == 'de':
        # replace claims references to [CLAIMS]
        # claims 1 to 2 --> [CLAIMS]
        # claim = re.sub(r'Anspr[u|ü]ch[e|en]{0,}\s\d{1,}\s[a-z]{1,}\s\d{1,}', '', claim)
        # # claim 1 to 2 --> [CLAIMS]
        # # i = re.sub(r'Ansprüche\s\d{1,}\s[a-z]{1,}\s\d{1,}', r'[CLAIMS]', i)
        # # i = re.sub(r'Ansprüchen\s\d{1,}\s[a-z]{1,}\s\d{1,}', r'[CLAIMS]', i)
        # # claim 1 --> [CLAIMS]
        # claim = re.sub(r'Anspruch\s\d{1,}', '', claim)
        
        # split compound German words
        # claim = doc_split.doc_split(claim)
        # claim = re.sub('·', ' ', claim)
        
        for s in c_s:
            if s in incorrect_words_de.keys():
                # print(s)
                claim = re.sub(s, incorrect_words_de[s], claim)        
    
    # replace groups of numbers with NUM
    # eg. 100, 200 --> NUM
    claim = re.sub(r'([0-9]{1,},{1,}\s{0,})+[0-9]{1,}', 'NUM ', claim)
    claim = re.sub(r'([a-zA-Z\']{0,}[0-9]{1,}[,;:]{1,}\s{0,})+[a-zA-Z\']{0,}[0-9]{1,}', 'NUM ', claim)
    # replace single number to NUM
    # eg. 1 --> NUM
    claim = re.sub(r'[0-9]{1,}', 'NUM', claim)        
    
    # replace contents within parantheses as [ABBRV]
    claim = re.sub(r'\(.+?\)', '', claim)
    
    # replace newline characters with space
    claim = re.sub(r'\n', ' ', claim)
    
    # replace punctuations with space
    # claim = re.sub(r'(\w)([!"#$%&\'()*+,-./:;<=>?@\[\\\\\]^_`{|}~])(\w)', '\1 \3', claim)
    
    claim = re.sub(r'([!"#$%&\'()*+,-./:;<=>?@\[\\\\\]^_`{|}~])', r' \1 ', claim)
    
    # claim = re.sub(r'([!"#$%&\'()*+,-./:;<=>?@\[\\\\\]^`{|}~])', ' ', claim)
    claim = re.sub(r'([a-zA-Z])-\s', r'\1 -', claim)
    claim = re.sub(r'[^a-zA-Z][-\']', r' ', claim)
    
    claim = re.sub(r'\(([a-zA-Z\s]{0,}NUM[a-zA-Z\s]{0,})+\)', 'NUM', claim)
    
    # replace double spaces with single space
    claim = re.sub(r'\s{1,}', ' ', claim)    
    
    return claim

In [21]:
WORDS_TO_OMIT = ['NUM', 'claim', 'Claim', 
                'anspr', 'Anspr']
def find_and_replace_ncs(claim, lang):    
    nlp = nlp_en if lang=='en' else nlp_de
    doc = nlp(claim)
    for nc in doc.noun_chunks:
        comp_ = []
        for token in nc:
            if token.pos_ not in ['DET', 'ADV', 'NUM', 'CCONJ'] and \
               token.dep_ not in ['poss'] and \
               token.text not in ['said', 'more', 'following', 'comprising'] and \
               not token.is_punct:
                comp_.append(token.text)
        comp_ = ' '.join(comp_)
        comp_ = re.sub(r'(\s\s)+', '', comp_)
        if comp_:
            if len(comp_.split(' ')) > 1:
                omit_flag = False
                for word in WORDS_TO_OMIT:
                    if word in comp_:
                        omit_flag = True
                        break
                if not omit_flag:
                    # rep = re.sub('-', '', comp_)
                    rep = re.sub(r'\s{1,}', '_', comp_)
                    # print('comp_', comp_)
                    claim = re.sub(comp_, rep, claim)
    return claim

## Define evaluate function

In [22]:
def evaluate(encoding, num_ret, en_doc_vectors, de_doc_vectors): 
    prec_av, recall_av, fscore_av = 0, 0, 0
    
    for query, own_claims_en, own_claims_de, \
        rel_claims_en, rel_claims_de in zip(test_final.query_title,
                                            test_final.own_claims_en,
                                            test_final.own_claims_de,
                                            test_final.rel_claims_en,
                                            test_final.rel_claims_de):
        
        # print('query', query)
        
        relevant_claims = own_claims_en + own_claims_de + \
                            rel_claims_en + rel_claims_de
        
        # num_ret = int(len(relevant_claims)/2)
        # num_ret = 50
        out_en = get_retrieved_docs_test(
                                      encoding=encoding,
                                      query=query, 
                                      num_ret=num_ret, 
                                      docs=all_claims_en, 
                                      doc_vectors=en_doc_vectors, 
                                      lang='en')
        
        out_de = get_retrieved_docs_test(
                                      encoding=encoding,
                                      query=query, 
                                      num_ret=num_ret, 
                                      docs=all_claims_de, 
                                      doc_vectors=de_doc_vectors, 
                                      lang='en')
        
        retrieved_claims = out_en['docs'] + out_de['docs']
        
        relevant_retrieved = 0
        for claim in relevant_claims:
            if claim in retrieved_claims:
                relevant_retrieved += 1
        
        relevant = len(relevant_claims)
        retreived = len(retrieved_claims)
        
        prec = relevant_retrieved/retreived
        recall = relevant_retrieved/relevant     
    
        # print('P: %f R: %f' %(prec, recall))
    
        prec_av += prec
        recall_av += recall
    
    print('-- CLIR TASK --')
    print('P: %f R: %f' %(prec_av/len(test_small_df.query_title), 
                                recall_av/len(test_small_df.query_title)))

In [23]:
def evaluate_sent_similarity(en_doc_vectors, de_doc_vectors):
    idx = 0
    count_en_de, count_de_en = 0, 0
    for en, de, en_vec, de_vec in zip(all_claims_en, 
                                      all_claims_de,
                                      en_doc_vectors,
                                      de_doc_vectors):
        output_en_de = F.cosine_similarity(en_vec.unsqueeze(0), de_doc_vectors, dim=1)
        ret = torch.topk(output_en_de, 1).indices.cpu().numpy()
        if idx in ret:
            count_en_de += 1
            
            
        output_de_en = F.cosine_similarity(de_vec.unsqueeze(0), en_doc_vectors, dim=1)
        ret = torch.topk(output_de_en, 1).indices.cpu().numpy()
        if idx in ret:
            count_de_en += 1            

        idx += 1

    print('-- SENTENCE SIMILARITY --')
    print('EN - DE Accuracy: %f' %(count_en_de/len(all_claims_en)))
    print('DE - EN Accuracy: %f' %(count_de_en/len(all_claims_de)))

In [24]:
import re
def evaluate_word_similarity(m_type, model, vocab):
    if m_type=='word2vec':
        sim_fn = model.wv.most_similar
    elif m_type=='fasttext':
        sim_fn = model.similar_by_word

    total_en_de, total_de_en, count_en_de, count_de_en = 0, 0, 0, 0
    for en, de in zip(algn_dic.en[:100],
                      algn_dic.de[:100]):
        en = re.sub('_', ' ', en)
        de = re.sub('_', ' ', de)
        if en in vocab:
            total_en_de += 1
            sim_words = sim_fn(en, topn=10)
            words = [sw[0] for sw in sim_words]
            if de in words:
                count_en_de += 1

        if de in vocab:
            total_de_en += 1
            sim_words = sim_fn(de, topn=10)
            words = [sw[0] for sw in sim_words]
            if en in words:
                count_de_en += 1        
                
    print('-- WORD SIMILARITY --')
    print('EN - DE Accuracy:', count_en_de/total_en_de)
    print('DE - EN Accuracy:', count_de_en/total_de_en)

In [25]:
def get_word_vec():
    en_vec = []
    for word in algn_dic.en[:100]:
        word = re.sub('_', ' ', word)
        vec = encoding.encode(word, 'en')
        if vec.shape == torch.Size([]):
            vec = torch.zeros(300)
        en_vec.append(vec)

    de_vec = []
    for word in algn_dic.de[:100]:
        word = re.sub('_', ' ', word)
        vec = encoding.encode(word, 'de')
        if vec.shape == torch.Size([]):
            vec = torch.zeros(300)    
        de_vec.append(vec)    

    en_vec = torch.stack(en_vec)
    de_vec = torch.stack(de_vec)
    
    return en_vec, de_vec

In [26]:
import re
def evaluate_word_similarity_post_hoc(encoding):
    en_vec, de_vec = get_word_vec()
    idx = 0
    total_en_de, total_de_en, count_en_de, count_de_en = 0, 0, 0, 0
    for en, de, e_vec, d_vec in zip(algn_dic.en[:100],
                                      algn_dic.de[:100],
                                      en_vec,
                                      de_vec
                                     ):
        en = re.sub('_', ' ', en)
        de = re.sub('_', ' ', de)
        
        output = F.cosine_similarity(e_vec.unsqueeze(0), de_vec, dim=1)
        ret_ids = torch.topk(output, 10).indices.cpu().numpy()
        if idx in ret_ids:
            count_en_de += 1

        output = F.cosine_similarity(d_vec.unsqueeze(0), en_vec, dim=1)
        ret_ids = torch.topk(output, 10).indices.cpu().numpy() 
        if idx in ret_ids:
            count_de_en += 1
            
        idx += 1
                
    print('-- WORD SIMILARITY --')
    total_en_de, total_de_en = len(en_vec), len(de_vec)
    print('EN - DE Accuracy:', count_en_de/total_en_de)
    print('DE - EN Accuracy:', count_de_en/total_de_en)

# With alignment

## Word2Vec models

In [27]:
from gensim.models import Word2Vec as W2V

In [32]:
model = W2V.load("../../data/word2vec/word2vec.model")

11:27:34: loading Word2Vec object from ../../data/word2vec/word2vec.model
11:27:34: loading wv recursively from ../../data/word2vec/word2vec.model.wv.* with mmap=None
11:27:34: loading vectors from ../../data/word2vec/word2vec.model.wv.vectors.npy with mmap=None
11:27:34: loading syn1neg from ../../data/word2vec/word2vec.model.syn1neg.npy with mmap=None
11:27:34: setting ignored attribute cum_table to None
11:27:36: Word2Vec lifecycle event {'fname': '../../data/word2vec/word2vec.model', 'datetime': '2022-03-08T11:27:36.860523', 'gensim': '4.1.2', 'python': '3.8.10 (default, Nov 26 2021, 20:14:08) \n[GCC 9.3.0]', 'platform': 'Linux-5.4.0-92-generic-x86_64-with-glibc2.29', 'event': 'loaded'}


In [33]:
wv_vocab = []
for idx in range(len(model.wv)):
    wv_vocab.append(model.wv.index_to_key[idx])
wv_vocab = wv_vocab
len(wv_vocab)

306606

In [60]:
encoding = Encoding(model=model,
                   vocab=wv_vocab,
                   do_pre_process=True,
                   do_noun_chunks=True,
                   do_dual_token=True,
                   do_dual_embed=True,
                   do_avg=True)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity(m_type='word2vec',
                        model=model,
                        vocab=wv_vocab)

KeyboardInterrupt: 

In [91]:
encoding = Encoding(model=model,
                   vocab=wv_vocab,
                   do_pre_process=True,
                   do_noun_chunks=True,
                   do_dual_token=True,
                   do_dual_embed=True,
                   do_avg=False)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity(m_type='word2vec',
                        model=model,
                        vocab=wv_vocab)


-- CLIR TASK --
P: 0.198200 R: 0.519148
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.187883
DE - EN Accuracy: 0.271472
-- WORD SIMILARITY --
EN - DE Accuracy: 0.25
DE - EN Accuracy: 0.3333333333333333


In [92]:
encoding = Encoding(model=model,
                   vocab=wv_vocab,
                   do_pre_process=True,
                   do_noun_chunks=True,
                   do_dual_token=True,
                   do_dual_embed=False,
                   do_avg=False)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity(m_type='word2vec',
                        model=model,
                        vocab=wv_vocab)


-- CLIR TASK --
P: 0.198200 R: 0.514256
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.134202
DE - EN Accuracy: 0.229294
-- WORD SIMILARITY --
EN - DE Accuracy: 0.25
DE - EN Accuracy: 0.3333333333333333


In [93]:
encoding = Encoding(model=model,
                   vocab=wv_vocab,
                   do_pre_process=True,
                   do_noun_chunks=True,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=False)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity(m_type='word2vec',
                        model=model,
                        vocab=wv_vocab)


-- CLIR TASK --
P: 0.177800 R: 0.497271
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.105061
DE - EN Accuracy: 0.173313
-- WORD SIMILARITY --
EN - DE Accuracy: 0.25
DE - EN Accuracy: 0.3333333333333333


In [94]:
encoding = Encoding(model=model,
                   vocab=wv_vocab,
                   do_pre_process=True,
                   do_noun_chunks=False,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=False)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity(m_type='word2vec',
                        model=model,
                        vocab=wv_vocab)


-- CLIR TASK --
P: 0.177800 R: 0.497271
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.105061
DE - EN Accuracy: 0.173313
-- WORD SIMILARITY --
EN - DE Accuracy: 0.25
DE - EN Accuracy: 0.3333333333333333


In [95]:
encoding = Encoding(model=model,
                   vocab=wv_vocab,
                   do_pre_process=False,
                   do_noun_chunks=False,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=False)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity(m_type='word2vec',
                        model=model,
                        vocab=wv_vocab)


-- CLIR TASK --
P: 0.177800 R: 0.497271
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.105061
DE - EN Accuracy: 0.173313
-- WORD SIMILARITY --
EN - DE Accuracy: 0.25
DE - EN Accuracy: 0.3333333333333333


## Fasttext models

In [74]:
from gensim.models import FastText

In [2]:
import numpy as np

In [4]:
model = FastText.load('../../data/fasttext/epo_w_ncs_fasttext.model')

In [7]:
wv_vocab = []
for idx in tq(range(len(model.wv))):
    wv_vocab.append(model.wv.index_to_key[idx])
len(wv_vocab)

745335

In [36]:
encoding = Encoding(model=model,
                   vocab=wv_vocab,
                   do_pre_process=True,
                   do_noun_chunks=True,
                   do_dual_token=True,
                   do_dual_embed=True,
                   do_avg=True)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity(m_type='word2vec',
                        model=model,
                        vocab=wv_vocab)


-- CLIR TASK --
P: 0.212000 R: 0.549574
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.222393
DE - EN Accuracy: 0.071319


AttributeError: 'DataFrame' object has no attribute 'dic'

In [107]:
encoding = Encoding(model=model,
                   vocab=wv_vocab,
                   do_pre_process=True,
                   do_noun_chunks=True,
                   do_dual_token=True,
                   do_dual_embed=True,
                   do_avg=False)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity(m_type='word2vec',
                        model=model,
                        vocab=wv_vocab)


-- CLIR TASK --
P: 0.218000 R: 0.561513
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.226227
DE - EN Accuracy: 0.088190
-- WORD SIMILARITY --
EN - DE Accuracy: 0.11764705882352941
DE - EN Accuracy: 0.017857142857142856


In [108]:
encoding = Encoding(model=model,
                   vocab=wv_vocab,
                   do_pre_process=True,
                   do_noun_chunks=True,
                   do_dual_token=True,
                   do_dual_embed=False,
                   do_avg=False)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity(m_type='word2vec',
                        model=model,
                        vocab=wv_vocab)


-- CLIR TASK --
P: 0.207400 R: 0.530817
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.208589
DE - EN Accuracy: 0.066718
-- WORD SIMILARITY --
EN - DE Accuracy: 0.11764705882352941
DE - EN Accuracy: 0.017857142857142856


In [109]:
encoding = Encoding(model=model,
                   vocab=wv_vocab,
                   do_pre_process=True,
                   do_noun_chunks=True,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=False)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity(m_type='word2vec',
                        model=model,
                        vocab=wv_vocab)


-- CLIR TASK --
P: 0.204200 R: 0.526455
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.167945
DE - EN Accuracy: 0.065184
-- WORD SIMILARITY --
EN - DE Accuracy: 0.11764705882352941
DE - EN Accuracy: 0.017857142857142856


In [110]:
encoding = Encoding(model=model,
                   vocab=wv_vocab,
                   do_pre_process=True,
                   do_noun_chunks=False,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=False)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity(m_type='word2vec',
                        model=model,
                        vocab=wv_vocab)


-- CLIR TASK --
P: 0.204200 R: 0.526455
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.167945
DE - EN Accuracy: 0.065184
-- WORD SIMILARITY --
EN - DE Accuracy: 0.11764705882352941
DE - EN Accuracy: 0.017857142857142856


In [111]:
encoding = Encoding(model=model,
                   vocab=wv_vocab,
                   do_pre_process=False,
                   do_noun_chunks=False,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=False)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity(m_type='word2vec',
                        model=model,
                        vocab=wv_vocab)


-- CLIR TASK --
P: 0.204200 R: 0.526455
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.167945
DE - EN Accuracy: 0.065184
-- WORD SIMILARITY --
EN - DE Accuracy: 0.11764705882352941
DE - EN Accuracy: 0.017857142857142856


# Post-hoc alignment

In [28]:
def get_vec_from_txt(path):
    file = open(path, 'r')

    word_vectors = {}
    ind = 0
    line = file.readline()
    while True:
        line = file.readline()
        if not line:
            break

        line_split = line.split(' ')
        word = line_split[0]
        vec = line_split[1:]

        vec_float = []
        for i in vec:
            vec_float.append(float(i))

        word_vectors[word] = vec_float

        # print('ind: %s'%ind, end="\r")

        ind += 1
        
    file.close()
    print(ind)
    
    return word_vectors

## Word2Vec models

In [53]:
en_wv_unsup = get_vec_from_txt('../../data/word2vec_post_hoc/vectors-en-unsup.txt')
de_wv_unsup = get_vec_from_txt('../../data/word2vec_post_hoc/vectors-de-unsup.txt')

16591
45266


In [54]:
en_vocab_unsup = list(en_wv_unsup.keys())
de_vocab_unsup = list(de_wv_unsup.keys())

In [55]:
en_wv_sup = get_vec_from_txt('../../data/word2vec_post_hoc/vectors-en.txt')
de_wv_sup = get_vec_from_txt('../../data/word2vec_post_hoc/vectors-de.txt')

16591
45266


In [56]:
en_vocab_sup = list(en_wv_sup.keys())
de_vocab_sup = list(de_wv_sup.keys())

In [61]:
# SUP, Avg

encoding = Encoding(model=[en_wv_sup, de_wv_sup],
                   vocab=[en_vocab_sup, de_vocab_sup],
                   do_pre_process=False,
                   do_noun_chunks=False,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=True,
                   post_hoc_encode=True)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity_post_hoc(encoding=encoding)


-- CLIR TASK --
P: 0.202000 R: 0.564079
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.207822
DE - EN Accuracy: 0.286043
-- WORD SIMILARITY --
EN - DE Accuracy: 0.63
DE - EN Accuracy: 0.64


<ipython-input-13-00553723716c>:78: RuntimeWarning: invalid value encountered in double_scalars
  sent_vector = torch.tensor(np.sum(word_vectors, axis=0)/len(word_vectors))


In [62]:
# SUP, Sum

encoding = Encoding(model=[en_wv_sup, de_wv_sup],
                   vocab=[en_vocab_sup, de_vocab_sup],
                   do_pre_process=False,
                   do_noun_chunks=False,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=False,
                   post_hoc_encode=True)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity_post_hoc(encoding=encoding)


-- CLIR TASK --
P: 0.202000 R: 0.564079
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.207822
DE - EN Accuracy: 0.286043
-- WORD SIMILARITY --
EN - DE Accuracy: 0.63
DE - EN Accuracy: 0.64


In [63]:
# UNSUP, Avg

encoding = Encoding(model=[en_wv_unsup, de_wv_unsup],
                   vocab=[en_vocab_unsup, de_vocab_unsup],
                   do_pre_process=False,
                   do_noun_chunks=False,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=True,
                   post_hoc_encode=True)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity_post_hoc(encoding=encoding)


-- CLIR TASK --
P: 0.207800 R: 0.593201
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.284509
DE - EN Accuracy: 0.338957
-- WORD SIMILARITY --
EN - DE Accuracy: 0.64
DE - EN Accuracy: 0.63


<ipython-input-13-00553723716c>:78: RuntimeWarning: invalid value encountered in double_scalars
  sent_vector = torch.tensor(np.sum(word_vectors, axis=0)/len(word_vectors))


In [64]:
# UNSUP, Sum

encoding = Encoding(model=[en_wv_unsup, de_wv_unsup],
                   vocab=[en_vocab_unsup, de_vocab_unsup],
                   do_pre_process=False,
                   do_noun_chunks=False,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=False,
                   post_hoc_encode=True)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity_post_hoc(encoding=encoding)


-- CLIR TASK --
P: 0.207800 R: 0.593201
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.284509
DE - EN Accuracy: 0.338957
-- WORD SIMILARITY --
EN - DE Accuracy: 0.64
DE - EN Accuracy: 0.63


## FastText models

In [29]:
# Get wv from sup MUSE

en_wv_sup = get_vec_from_txt('../../data/fastText_post_hoc/vectors-en.txt')
de_wv_sup = get_vec_from_txt('../../data/fastText_post_hoc/vectors-de.txt')

97354
392292


In [30]:
en_vocab_sup = list(en_wv_sup.keys())
de_vocab_sup = list(de_wv_sup.keys())

In [31]:
# Get wv from unsup MUSE

en_wv_unsup = get_vec_from_txt('../../data/fastText_post_hoc/vectors-en-unsup.txt')
de_wv_unsup = get_vec_from_txt('../../data/fastText_post_hoc/vectors-de-unsup.txt')

97354
392292


In [32]:
en_vocab_unsup = list(en_wv_unsup.keys())
de_vocab_unsup = list(de_wv_unsup.keys())

In [69]:
# SUP, Avg

encoding = Encoding(model=[en_wv_sup, de_wv_sup],
                   vocab=[en_vocab_sup, de_vocab_sup],
                   do_pre_process=False,
                   do_noun_chunks=False,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=True,
                   post_hoc_encode=True)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity_post_hoc(encoding=encoding)


-- CLIR TASK --
P: 0.198200 R: 0.560173
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.231595
DE - EN Accuracy: 0.325153


<ipython-input-13-00553723716c>:78: RuntimeWarning: invalid value encountered in double_scalars
  sent_vector = torch.tensor(np.sum(word_vectors, axis=0)/len(word_vectors))


-- WORD SIMILARITY --
EN - DE Accuracy: 0.89
DE - EN Accuracy: 0.89


In [70]:
# SUP, Sum

encoding = Encoding(model=[en_wv_sup, de_wv_sup],
                   vocab=[en_vocab_sup, de_vocab_sup],
                   do_pre_process=False,
                   do_noun_chunks=False,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=False,
                   post_hoc_encode=True)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity_post_hoc(encoding=encoding)


-- CLIR TASK --
P: 0.198200 R: 0.560173
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.231595
DE - EN Accuracy: 0.325153
-- WORD SIMILARITY --
EN - DE Accuracy: 0.89
DE - EN Accuracy: 0.89


In [33]:
# UNSUP, Avg

encoding = Encoding(model=[en_wv_unsup, de_wv_unsup],
                   vocab=[en_vocab_unsup, de_vocab_unsup],
                   do_pre_process=False,
                   do_noun_chunks=False,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=True,
                   post_hoc_encode=True)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity_post_hoc(encoding=encoding)

KeyboardInterrupt: 

In [72]:
# UNSUP, Sum

encoding = Encoding(model=[en_wv_unsup, de_wv_unsup],
                   vocab=[en_vocab_unsup, de_vocab_unsup],
                   do_pre_process=False,
                   do_noun_chunks=False,
                   do_dual_token=False,
                   do_dual_embed=False,
                   do_avg=False,
                   post_hoc_encode=True)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

evaluate_word_similarity_post_hoc(encoding=encoding)


-- CLIR TASK --
P: 0.207600 R: 0.587086
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.291411
DE - EN Accuracy: 0.360429
-- WORD SIMILARITY --
EN - DE Accuracy: 0.9
DE - EN Accuracy: 0.89


# Contextual

## SentenceBERT

In [251]:
from sentence_transformers import SentenceTransformer

In [9]:
model = SentenceTransformer('../../../sentence-transformers/examples/training/multilingual/output/make-multilingual-sys-2021-09-17_16-10-50/')

In [73]:
encoding = Encoding(model=model,
                   contextual_encode=True)

In [52]:
en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

In [53]:
evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)

-- CLIR TASK --
P: 0.242400 R: 0.715028


In [55]:
evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)

-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.525307
DE - EN Accuracy: 0.520706


In [74]:
evaluate_word_similarity_post_hoc(encoding=encoding)

-- WORD SIMILARITY --
EN - DE Accuracy: 0.86
DE - EN Accuracy: 0.87


## Pre-Trained MLP + NSP

In [39]:
from transformers import BertTokenizer
import torch
import numpy as np

In [40]:
import sys
sys.path.append('../')
from util import util, corpus, alignment

from tqdm.notebook import tqdm as tq
import pandas as pd

import torch
import torch.nn.functional as F

In [41]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = torch.load('../Sentence-BERT/data/model_MLM_NSP.pt').module
model.eval()

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [51]:
def get_preds(query, docs, num_ret):
    preds = []
    for doc in tq(docs):
        with torch.no_grad():
            inputs = tokenizer(query, doc, return_tensors='pt',
                          max_length=512, padding=True, truncation=True).to('cuda')
            output = model(**inputs)
            pred = output.seq_relationship_logits.detach().cpu().numpy()[0]
            preds.append(pred)
            
    pred_isnext = [pred[0] for pred in preds]
    return get_top_k(pred_isnext, docs, num_ret)            

def get_top_k(preds, docs, num_ret):
    ind = np.argsort(preds)[-num_ret:][::-1]
    return [docs[i] for i in ind]

In [185]:
for i in range(12):
    print('With layer', i)
    encoding = Encoding(model=model,
                        contextual_encode_full=True,
                        layers=(i,i+1))
    en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)
    evaluate(
            encoding=encoding,
            num_ret=50,
            en_doc_vectors=en_doc_vectors,
            de_doc_vectors=de_doc_vectors)

    evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                            de_doc_vectors=de_doc_vectors)


    evaluate_word_similarity_post_hoc(encoding=encoding)
    print('------------')

With layer 0



-- CLIR TASK --
P: 0.097600 R: 0.262247
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.079755
DE - EN Accuracy: 0.178681
-- WORD SIMILARITY --
EN - DE Accuracy: 0.42
DE - EN Accuracy: 0.58
------------
With layer 1



-- CLIR TASK --
P: 0.114800 R: 0.292507
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.195552
DE - EN Accuracy: 0.252301
-- WORD SIMILARITY --
EN - DE Accuracy: 0.55
DE - EN Accuracy: 0.69
------------
With layer 2



-- CLIR TASK --
P: 0.109600 R: 0.275316
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.281442
DE - EN Accuracy: 0.328988
-- WORD SIMILARITY --
EN - DE Accuracy: 0.53
DE - EN Accuracy: 0.72
------------
With layer 3



-- CLIR TASK --
P: 0.101600 R: 0.241315
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.369632
DE - EN Accuracy: 0.378067
-- WORD SIMILARITY --
EN - DE Accuracy: 0.64
DE - EN Accuracy: 0.72
------------
With layer 4


...


-- CLIR TASK --
P: 0.077800 R: 0.174624
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.463190
DE - EN Accuracy: 0.480061
-- WORD SIMILARITY --
EN - DE Accuracy: 0.79
DE - EN Accuracy: 0.73
------------
With layer 8



-- CLIR TASK --
P: 0.076400 R: 0.176232
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.465491
DE - EN Accuracy: 0.473926
-- WORD SIMILARITY --
EN - DE Accuracy: 0.77
DE - EN Accuracy: 0.71
------------
With layer 9



-- CLIR TASK --
P: 0.077000 R: 0.185745
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.453988
DE - EN Accuracy: 0.464724
-- WORD SIMILARITY --
EN - DE Accuracy: 0.78
DE - EN Accuracy: 0.76
------------
With layer 10



-- CLIR TASK --
P: 0.074000 R: 0.178583
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.451687
DE - EN Accuracy: 0.467791
-- WORD SIMILARITY --
EN - DE Accuracy: 0.73
DE - EN Accuracy: 0.76
------------
With layer 11



-- CLIR TASK --
P: 0.083000 R: 0.194953
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.451687
DE - EN Accuracy: 0.454755
-- WORD SIMILARITY --
EN - DE Accuracy: 0.8
DE - EN Accuracy: 0.75
------------


In [68]:
for i in range(1, 14):
    print('With last %d layer(s): [-%d:13]' %(i,i))
    encoding = Encoding(model=model,
                        contextual_encode_full=True,
                        layers=(-i,13))
    en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)
    evaluate(
            encoding=encoding,
            num_ret=50,
            en_doc_vectors=en_doc_vectors,
            de_doc_vectors=de_doc_vectors)

    evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                            de_doc_vectors=de_doc_vectors)


    evaluate_word_similarity_post_hoc(encoding=encoding)

With last 1 layer(s): [-1:13]



-- CLIR TASK --
P: 0.136600 R: 0.363035
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.461656
DE - EN Accuracy: 0.445552
-- WORD SIMILARITY --
EN - DE Accuracy: 0.8
DE - EN Accuracy: 0.78
With last 2 layer(s): [-2:13]



-- CLIR TASK --
P: 0.103000 R: 0.251624
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.453988
DE - EN Accuracy: 0.466258
-- WORD SIMILARITY --
EN - DE Accuracy: 0.78
DE - EN Accuracy: 0.79
With last 3 layer(s): [-3:13]



-- CLIR TASK --
P: 0.087200 R: 0.214749
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.452454
DE - EN Accuracy: 0.470092
-- WORD SIMILARITY --
EN - DE Accuracy: 0.84
DE - EN Accuracy: 0.78
With last 4 layer(s): [-4:13]



-- CLIR TASK --
P: 0.095200 R: 0.224138
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.464724
DE - EN Accuracy: 0.479294
-- WORD SIMILARITY --
EN - DE Accuracy: 0.81
DE - EN Accuracy: 0.81
With last 5 layer(s): [-5:13]


...


-- CLIR TASK --
P: 0.084800 R: 0.206069
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.454755
DE - EN Accuracy: 0.465491
-- WORD SIMILARITY --
EN - DE Accuracy: 0.78
DE - EN Accuracy: 0.8
With last 10 layer(s): [-10:13]



-- CLIR TASK --
P: 0.089400 R: 0.208039
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.460123
DE - EN Accuracy: 0.475460
-- WORD SIMILARITY --
EN - DE Accuracy: 0.78
DE - EN Accuracy: 0.76
With last 11 layer(s): [-11:13]



-- CLIR TASK --
P: 0.089800 R: 0.213063
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.447853
DE - EN Accuracy: 0.467025
-- WORD SIMILARITY --
EN - DE Accuracy: 0.76
DE - EN Accuracy: 0.77
With last 12 layer(s): [-12:13]



-- CLIR TASK --
P: 0.092000 R: 0.213057
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.455521
DE - EN Accuracy: 0.458589
-- WORD SIMILARITY --
EN - DE Accuracy: 0.77
DE - EN Accuracy: 0.77
With last 13 layer(s): [-13:13]



-- CLIR TASK --
P: 0.089400 R: 0.213343
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.457822
DE - EN Accuracy: 0.462423
-- WORD SIMILARITY --
EN - DE Accuracy: 0.76
DE - EN Accuracy: 0.76


In [32]:
def evaluate_cont_pre(num_ret, en_docs, de_docs): 
    prec_av, recall_av, fscore_av = 0, 0, 0
    
    for query, own_claims_en, own_claims_de, \
        rel_claims_en, rel_claims_de in zip(tq(test_final.query_title),
                                            test_final.own_claims_en,
                                            test_final.own_claims_de,
                                            test_final.rel_claims_en,
                                            test_final.rel_claims_de):
                
        relevant_claims = own_claims_en + own_claims_de + \
                            rel_claims_en + rel_claims_de
        
        out_en = get_preds(query, en_docs, num_ret)
        out_de = get_preds(query, de_docs, num_ret)
        
        retrieved_claims = out_en + out_de
        
        relevant_retrieved = 0
        for claim in relevant_claims:
            if claim in retrieved_claims:
                relevant_retrieved += 1
        
        relevant = len(relevant_claims)
        retreived = len(retrieved_claims)
        
        prec = relevant_retrieved/retreived
        recall = relevant_retrieved/relevant     
    
        # print('P: %f R: %f' %(prec, recall))
    
        prec_av += prec
        recall_av += recall
    
    print('-- CLIR TASK --')
    print('P: %f R: %f' %(prec_av/len(test_small_df.query_title), 
                                recall_av/len(test_small_df.query_title)))

In [41]:
evaluate_cont_pre(num_ret=50, en_docs=all_claims_en, de_docs=all_claims_de)



-- CLIR TASK --
P: 0.244800 R: 0.737299


In [65]:
def evaluate_sent_similarity_cont_pre():
    idx = 0
    count_en_de, count_de_en = 0, 0
    for en, de in zip(tq(all_claims_en[:2]), all_claims_de[:2]):
        output_en_de = get_preds(en, all_claims_de, 1)
        if output_en_de == de:
            count_en_de += 1
             
        output_de_en = get_preds(de, all_claims_en, 1)
        if output_de_en == en:
            count_de_en += 1            

        idx += 1

    print('-- SENTENCE SIMILARITY --')
    print('EN - DE Accuracy: %f' %(count_en_de/len(all_claims_en)))
    print('DE - EN Accuracy: %f' %(count_de_en/len(all_claims_de)))

In [66]:
evaluate_sent_similarity_cont_pre()


-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.000000
DE - EN Accuracy: 0.000000


In [81]:
def evaluate_word_similarity_cont_pre():
    idx = 0
    count_en_de, count_de_en = 0, 0
    for en, de in zip(tq(algn_dic.en[:100]),
                      algn_dic.de[:100],
                      ):
        en = re.sub('_', ' ', en)
        de = re.sub('_', ' ', de)
        
        output_en_de = get_preds(en, [re.sub('_', ' ', x) for x in algn_dic.de[:100]], 10)
        if de in output_en_de:
            count_en_de += 1

        output_de_en = get_preds(de, [re.sub('_', ' ', x) for x in algn_dic.en[:100]], 10)
        if en in output_de_en:
            count_de_en += 1
            
        idx += 1
                
    print('-- WORD SIMILARITY --')
    print('EN - DE Accuracy:', count_en_de/idx)
    print('DE - EN Accuracy:', count_de_en/idx)

In [82]:
evaluate_word_similarity_cont_pre()


-- WORD SIMILARITY --
EN - DE Accuracy: 0.88
DE - EN Accuracy: 0.9


## Static baseline

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
en_vectors = KeyedVectors.load_word2vec_format('../../../resources/test/fasttext_pre-trained/wiki.en.align.vec', 
                                               binary=False)  # C text format

de_vectors = KeyedVectors.load_word2vec_format('../../../resources/test/fasttext_pre-trained/wiki.de.align.vec', 
                                               binary=False)  # C text format

In [ ]:
en_vocab = []
for i in tq(range(len(en_vectors))):
    en_vocab.append(en_vectors.index_to_key[i])
    
de_vocab = []
for i in tq(range(len(de_vectors))):
    de_vocab.append(de_vectors.index_to_key[i])

In [ ]:
import numpy as np
import torch

def encode(sent, lang):
    
    model = en_vectors if lang=='en' else de_vectors
    vocab = en_vocab if lang=='en' else de_vocab
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if torch.cuda.is_available():
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    
    # TODO: vector dimension is hardcoded. Change that.
    # sent_vector = torch.zeros([300], dtype=torch.float32, device=device)
    
    tokens = tokenize.get_tokens(sent, lang=lang)
    # tokens = sent.split(' ')
    word_vectors = [np.asarray(model[token]) for token in tokens if token in vocab]
    sent_vector = torch.tensor(np.sum(word_vectors, axis=0))
    

    return sent_vector

In [ ]:
en_doc_vectors = []
for claim in tq(all_claims_en):
    en_doc_vectors.append(encode(claim, lang='en'))

In [ ]:
de_doc_vectors = []
for claim in tq(all_claims_de):
    de_doc_vectors.append(encode(claim, lang='de'))

In [ ]:
en_doc_vectors = torch.stack(en_doc_vectors)
de_doc_vectors = torch.stack(de_doc_vectors)

In [ ]:
evaluate()

## Contextual baseline

In [252]:
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

20:24:16: Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
20:24:17: Lock 140416906847952 acquired on /home/sudhi/.cache/torch/sentence_transformers/sentence-transformers__paraphrase-multilingual-mpnet-base-v2.fa41ae18c0ef9289595fcf78757446e10ed6a492/.gitattributes.lock


20:24:18: Lock 140416906847952 released on /home/sudhi/.cache/torch/sentence_transformers/sentence-transformers__paraphrase-multilingual-mpnet-base-v2.fa41ae18c0ef9289595fcf78757446e10ed6a492/.gitattributes.lock


20:24:18: Lock 140416906849584 acquired on /home/sudhi/.cache/torch/sentence_transformers/sentence-transformers__paraphrase-multilingual-mpnet-base-v2.fa41ae18c0ef9289595fcf78757446e10ed6a492/README.md.lock


20:24:19: Lock 140416906849584 released on /home/sudhi/.cache/torch/sentence_transformers/sentence-transformers__paraphrase-multilingual-mpnet-base-v2.fa41ae18c0ef9289595fcf78757446e10ed6a492/README.md.lock


20:24:19: Lock 140416906849584 acquired on /home/sudhi/.cache/torch/sentence_transformers/sentence-transformers__paraphrase-multilingual-mpnet-base-v2.fa41ae18c0ef9289595fcf78757446e10ed6a492/config.json.lock


...

20:24:39: Lock 140418544362736 acquired on /home/sudhi/.cache/torch/sentence_transformers/sentence-transformers__paraphrase-multilingual-mpnet-base-v2.fa41ae18c0ef9289595fcf78757446e10ed6a492/tokenizer_config.json.lock


20:24:39: Lock 140418544362736 released on /home/sudhi/.cache/torch/sentence_transformers/sentence-transformers__paraphrase-multilingual-mpnet-base-v2.fa41ae18c0ef9289595fcf78757446e10ed6a492/tokenizer_config.json.lock


20:24:40: Lock 140416905948512 acquired on /home/sudhi/.cache/torch/sentence_transformers/sentence-transformers__paraphrase-multilingual-mpnet-base-v2.fa41ae18c0ef9289595fcf78757446e10ed6a492/1_Pooling/config.json.lock


20:24:40: Lock 140416905948512 released on /home/sudhi/.cache/torch/sentence_transformers/sentence-transformers__paraphrase-multilingual-mpnet-base-v2.fa41ae18c0ef9289595fcf78757446e10ed6a492/1_Pooling/config.json.lock


20:24:43: Use pytorch device: cuda


In [77]:
encoding = Encoding(model=model,
                   contextual_encode=True)

en_doc_vectors, de_doc_vectors = vectorize_docs(encoding=encoding)

evaluate(
        encoding=encoding,
        num_ret=50,
        en_doc_vectors=en_doc_vectors,
        de_doc_vectors=de_doc_vectors)


evaluate_sent_similarity(en_doc_vectors=en_doc_vectors,
                        de_doc_vectors=de_doc_vectors)


evaluate_word_similarity_post_hoc(encoding=encoding)


-- CLIR TASK --
P: 0.205600 R: 0.637138
-- SENTENCE SIMILARITY --
EN - DE Accuracy: 0.419479
DE - EN Accuracy: 0.462423
-- WORD SIMILARITY --
EN - DE Accuracy: 0.85
DE - EN Accuracy: 0.8
